In [1]:
from flask import Flask,jsonify,render_template
from flask_cors import CORS
import json
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import datetime as dt

make flask server and do sql stuff

In [27]:
app = Flask(__name__)
CORS(app)
with open("../restaurant_data.json", 'r') as file:
    data = json.load(file)

restaurnt_df = pd.DataFrame(data)

engine = create_engine("sqlite:///Resources/votes.sqlite")
base = automap_base()
base.prepare(autoload_with=engine)
types = base.classes.Favorite_Cuisines

session = Session(bind=engine)
session.add("something")
session.add(types(restaurant_type = "mexican", votes_cast = 0))
session.add(types(restaurant_type = "italian", votes_cast = 0))
session.add(types(restaurant_type = "thai", votes_cast = 0))
session.close()

def pull_vote_totals():
    print("pulling vote totals")
    session = Session(bind=engine)
    boop = session.query(types).all()
    democracy = {"mexican":0, "italian": 0, "thai":0}
    
    for i in boop:
        print(i.restaurant_type)
        democracy[i.restaurant_type] = i.votes_cast   
        print(i.votes_cast) 
    session.close()
    print("pulled vote totals")
    print(democracy)
    return democracy
    

def add_vote_to_db(vote):
    print("adding votes to db")
    session = Session(bind=engine)
    
    tp = session.query(types).filter(types.restaurant_type == vote)
    for t in tp:
        print("looping through tp")
        print(t.votes_cast)
        t.votes_cast += 1
        print(t.votes_cast)
    session.commit()
    print(f"added {vote} to db")
    session.close()
    return json.dumps("vote added to db")
    

@app.route("/")
def home():
    return render_template('index.html')

@app.route("/map")
def dontwait():
    return render_template('map.html')

@app.route("/chart")
def justkiddingyoucanwait():
    return render_template('chart.html')

@app.route("/btc")
def nvm():
    return render_template('btc.html')

@app.route("/vote<vote>")
def noway(vote):
    add_vote_to_db(vote)
    return vote
    
@app.route("/voteTot")
def jose():
    voteTots = pull_vote_totals()
    print(voteTots)
    return json.dumps(voteTots)


if(__name__=="__main__"):
    app.run()

UnmappedInstanceError: Class 'builtins.str' is not mapped